# MAT281

## Aplicaciones de la Matemática en la Ingeniería

Puedes ejecutar este jupyter notebook de manera interactiva:

[![Binder](../shared/images/jupyter_binder.png)](https://mybinder.org/v2/gh/sebastiandres/mat281_m02_introduccion/master?filepath=05_analisis_datos/05_analisis_datos.ipynb)

[![Colab](../shared/images/jupyter_colab.png)](https://colab.research.google.com/github/sebastiandres/mat281_m02_analisis_datos/blob/master//05_analisis/05_analisis.ipynb)

## ¿Qué contenido aprenderemos?
* Exploración de datos.
* Eliminando datos.
* Agrupando datos en pandas.

## ¿Porqué aprenderemos eso?

**1**. Complementa las otras tareas aprendidas: cargar datos de diversos formatos, pivotear tablas, usar merges, realizar SQL queries.

**2**. Resultan funcionalidades extremadamente comunes en la exploración de datos.

## ¿Cómo lo aprenderemos?

Tomaremos un set de datos interesante para trabajar: el ejemplo de [Zoo Data Set](http://archive.ics.uci.edu/ml/datasets/zoo) creado por Richard Forsyth, que revisamos en una clase anterior y de la cual no sacamos la mayor información.

In [ ]:
%ls data/

In [ ]:
%cat data/zoo.names

In [ ]:
import pandas as pd
import os
df = pd.read_csv(os.path.join("data","zoo.data"), sep=";")
df.head()

¿Que pasó?

¡Por eso siempre conviene mirar **bien** los archivos antes de abrirlos!

In [ ]:
%cat data/zoo.data

In [ ]:
import pandas as pd
import os
my_cols = ["animal_name", "hair", "feathers", "eggs", "milk", 
           "airborne", "aquatic", "predator", "toothed", "backbone", 
           "breathes", "venomous", "fins", "legs", "tail", "domestic", 
           "catsize", "type"]

df = pd.read_csv(os.path.join("data","zoo.data"), header=None, names=my_cols, sep=",")
df.head()

Como ya debería ser conocido, el método `describe` nos entrega un útil resumen de la información:

In [ ]:
df.describe()

Para tener más información, es posible indicar que se incluyan todas la columnas (numéricas y no numéricas):

In [ ]:
df.describe(include="all")

Personalmente, siempre ejecuto el comando de la siguiente manera:

```python
df.describe(include="all").fillna("").T
```

* Incluye todas las columnas en el análisis.
* Reemplaza los nans por strings vacíos para facilitar lectura.
* Pivotea el resultado para poder mirar todas las columnas fácilmente.

In [ ]:
df.describe(include="all").fillna("").T

Observamos que el animal "frog" viene en 2 ocasiones. 

En efecto, lo dice en la descripción del dataset. Una vez más, ¡conviene leer las condiciones en las que viene el dataset!

In [ ]:
df[df.animal_name=="frog"]

### Eliminando datos

Para eliminar filas específicas existen varias formas, de la cual conviene elegir la más sencilla según el contexto:
1. `drop_duplicates`
2. `drop`
3. Máscaras y relaciones booleanas.

1.- El problema anterior se puede resolver de manera sencilla utilizando el método `drop_duplicates` que elimina las columnas que son idénticas.

In [ ]:
df2 = df.drop_duplicates()
df2[20:30]

El caso anterior no funciona porque las filas no son perfectamente idénticas. En efecto, difieren en si son venenosas o no. Podemos pasarle más argumentos a drop_duplicates para lograr el objetivo.

In [ ]:
col_subset = ["animal_name", "hair", "feathers"]
df2 = df.drop_duplicates(subset=col_subset)
df2[20:30]

Si prestamos cuidadosa atención, observaremos que la indexación no ha cambiado, y que simplemente se han saltado el número 26. Si deseamos que la indexación sea re-enumerada, es posible utilizar el método `reset_index`. 

In [ ]:
col_subset = ["animal_name", "hair", "feathers"]
df2 = df.drop_duplicates(subset=col_subset).reset_index()
df2[20:30]

2.- Si queremos eliminar algunas filas específicas que conocemos por su numeración, podemos utilizar el método `drop`:

In [ ]:
df2 = df.drop(index=26)
df2[20:30]

In [ ]:
df2 = df.drop(index=26).reset_index()
df2[20:30]

El método `drop` también permite eliminar columnas:

In [ ]:
df2 = df.drop(columns=["domestic", "catsize", "type"])
df2[20:30]

### 3. Usando máscaras (masks) y relaciones booleandas

Se denomina máscaras (masks) a listas con elementos booleanos del mismo tamaño del arreglo o DataFrame original. Su uso resulta común en algorítmica y numpy, y también se extienden a los Dataframes.

In [ ]:
Por ejemplo, si quisiéramos eliminar todos los animales de tipo que son acuáticos **o** tienen veneno, haríamos:

In [ ]:
# Mascara booleana con True para animales acuáticos
m_aquatic = df.aquatic == 1   
# Máscara booleana con True para animales venenosos
m_venomous = df.venomous == 1 
# Mascara booleana deseada
m = pd.np.logical_not(pd.np.logical_or(m_aquatic, m_venomous))
# Subselección
df[m]

**Pregunta**:
    
¿Cómo podríamos obtener el subconjunto de animales que tiene 2 patas y plumas, y es de tamaño de similar a un gato?

In [ ]:
# ?

In [ ]:
# Respuesta
m_2_legs = df.legs==2
m_has_feathers = df.feathers==1
m_catsize = df.catsize==1
m = pd.np.logical_and(pd.np.logical_and(m_2_legs, m_has_feathers), m_catsize)
df[m]

## Agrupando datos
Para agrupar y obtener estadísticas de los grupos generados usaremos el método `groupby`, que puede resultar un poco complejo pues tiene diversas formas de utilización.

Consideremos el caso de la columna "domestic" que puede tener únicamente valores `0` o `1`:

In [ ]:
df.domestic.unique()

Al agrupar utilizando `groupby` obtenemos lo siguiente:

In [ ]:
dg = df.groupby("domestic")

In [ ]:
type(dg)

In [ ]:
len(dg)

Sin embargo, no es posible acceder directamente a los elementos

In [ ]:
dg[0]

Lo que se ha generado es una partición del dataframe original, agrupando sobre los posibles valores de la columna "domestic".

Ahora es posible realizar diversos tipos de preguntas sobre dichas particiones. 

Por ejemplo, ¿Cuántos elementos tienen los grupos "domésticos" y "no domésticos"?

In [ ]:
dg.animal_name.count()

In [ ]:
dg.feathers.count()

También es posible obtener otro tipo de estadísticas: 
mean, median, min, max, sum, describe, count/size.

In [ ]:
dg.venomous.count()

In [ ]:
dg.venomous.sum()

In [ ]:
dg.venomous.mean()

En general, cuando se tienen preguntas simples estos métodos se encadenan para obtener una tabla de respuestas sencilla. 

Por supuesto, podemos utilizar más de una columna al mismo tiempo:

In [ ]:
df_count = df.groupby(["hair","feathers"]).animal_name.count()
df_count

Para tener una tabla "regular", donde se repitan los indices conviene usar nuevamente `reset_index`:

In [ ]:
df_count = df.groupby(["hair","feathers"]).animal_name.count().reset_index()
df_count

Lo anterior permite saber que en este dataset existen 38 animales sin pelo ni plumas, 20 animales sin pelo y con plumas, y 43 animales con pelo pero sin plumas. Pero no existen animales con pelo y con plumas.

Podemos complejizar ligeramente lo anterior si queremos realizar estadísticas sobre 2 columnas simultáneamente, por ejemplo, en lugar de simplemente contar los animales, queremos saber simultáneamente cuantos son venenosos y cuantos son predadores:

In [ ]:
df_count = df.groupby(["hair","feathers"])[["venomous","predator"]].sum().reset_index()
df_count

Sin embargo, en la notación anterior, resultaba necesario aplicar la misma operación (`sum`) a ambas columnas ("venomous" y "predator") y no podíamos aplicar funciones diferentes a cada columna.

Lo anterior se generaliza con el método `agg` que nos entrega plena responsabilidad de las columnas a utilizar y la función a aplicar. Pero a cambio tiene el costo que debemos explicitar un diccionario donde se pasan como llaves cada una de las columnas a analizar y como valores las funciones (de manera explícita) que se deben aplicar a la columna respectiva.

Por ejemplo, si quisiéramos calcular el promedio de pies de cada animal y contar los animales venenosos, podríamos realizarlo de la siguiente manera:

In [ ]:
df_mixed = df.groupby(["hair","feathers"]).agg({"legs":pd.np.mean,
                                                "venomous":len}).reset_index()
df_mixed

Lo anterior resulta extredamente conveniente, puesto que permite definir funciones arbitrarias de agrupación. Por ejemplo, contar los valores únicos (distintos) en una agrupación, o incluso agrupar dichas combinaciones.

In [ ]:
def my_custom_function(my_series):
    # [1,2,2,4,2,10,9,10,2] -> regresa la cantidad de elementos distintos, en este caso, 5
    my_value = my_series.nunique()
    return my_value 

df_mixed = df.groupby(["domestic"]).agg({"legs":my_custom_function}).reset_index()
df_mixed

In [ ]:
def my_custom_function(my_series):
    # [1,2,2,4,2,10,9,10,2] -> regresa la cantidad de elementos distintos, en este caso, 5
    my_value = ", ".join([str(x) for x in sorted(my_series.unique())])
    return my_value 

df_mixed = df.groupby(["domestic"]).agg({"legs":my_custom_function}).reset_index()
df_mixed

O donde se puedan concatenar de manera sencilla los nombres de los animales:

In [ ]:
def my_custom_function(my_series):
    # [1,2,2,4,2,10,9,10,2] -> regresa la cantidad de elementos distintos, en este caso, 5
    my_value = ", ".join([str(x) for x in sorted(my_series.unique())])
    return my_value 

df_mixed = df.groupby(["domestic","legs"]).agg({"animal_name":my_custom_function,
                                                "type":len}).reset_index()
df_mixed

**Pregunta:**

¿Cómo podríamos obtener una tabla con el promedio aritmético, promedio armónico, desviación estándar y valores únicos de patas, para animales que contienen la letra "e" en su nombre y los que no contienen la letra "e" en su nombre, pero tienen más de 0 pies?

In [ ]:
# Agregar columnas necesarias
df["e_in_name"] = df.animal_name.str.contains("e")
df["legs_arithmetic_mean"] = df["legs"]
df["legs_std"] = df["legs"]
df["legs_unique_values"] = df["legs"]
df["legs_harmonic_mean"] = df["legs"]
# Subselección
m = df["legs"]>0
cols = ["animal_name", "e_in_name", "legs_arithmetic_mean", "legs_std","legs_unique_values", "legs_harmonic_mean"]
df_aux = df.loc[m, cols]

In [ ]:
# Revisemos el dataframe
df_aux.head(10)

In [ ]:
# Crear las funciones complejas
def harmonic_mean(x_arr):
    """
    returns n / ( 1/ x[0] + ... + 1/ x[n-1]) 
    """
    n = x_arr.shape[0]
    inv_x_arr = [1./xi for xi in x_arr]
    return n / sum(inv_x_arr)

unique_values = lambda x: "[" + ", ".join([str(x_i) for x_i in sorted(x.unique())]) + "]"

In [ ]:
df_aux.groupby(["e_in_name"]).agg({
                                   "legs_unique_values":unique_values,
                                   "legs_arithmetic_mean":pd.np.mean,
                                   "legs_std":pd.np.std,
                                   "legs_harmonic_mean":harmonic_mean,    
})